In [1]:
import sys
sys.path.append('../') 
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from copy import deepcopy
from typing import Callable

# Now you can import Strat1
from quantutils.backtester_utils import calc_stats, main_backtester
from itertools import product, combinations, permutations
from Alphas.alphaminer import AlphaMiner, generate_param_combinations_and_multi_index

%load_ext autoreload
%autoreload 2

In [5]:
def clean_cols(df):
    dfc = df.copy()
    dfc.columns = [col[:3] for col in dfc.columns]
    return dfc

In [6]:
df_fx_spot = clean_cols(pd.read_excel('../Data/FXSpot.xlsx', index_col=0, parse_dates=True))
df_fx_tr = clean_cols(pd.read_excel('../Data/FXTR.xlsx', index_col=0, parse_dates=True))

In [10]:
df_dict = {}
df_dict['price'] = deepcopy(df_fx_spot)
df_dict['ret'] = deepcopy(df_fx_tr.pct_change(fill_method='pad'))



C:\Users\Wazir\AppData\Local\Temp\ipykernel_31856\1686854792.py:3: FutureWarning: The 'fill_method' and 'limit' keywords in DataFrame.pct_change are deprecated and will be removed in a future version. Call ffill before calling pct_change instead.
  df_dict['ret'] = deepcopy(df_fx_tr.pct_change(fill_method='pad'))


In [7]:
df_fx_tr

,IHR,ZAR,SGD,PLN,ISK,RUB,CHF,BRL,PEN,TWD,...,JPY,COP,ARS,CLP,DKK,GBP,THB,CAD,NOK,HUF
date,,,,,,,,,,,,,,,,,,,,,
2000-01-03,104.8794,103.8351,96.6450,92.9915,99.9032,81.7778,84.3049,75.4395,101.4467,104.2097,...,106.3850,96.8545,104.5725,95.0015,86.2477,98.6827,99.3928,105.6601,97.4555,94.7530
2000-01-04,104.9538,104.5632,96.6318,93.0895,100.5021,81.3550,84.8163,74.1681,101.4715,106.9662,...,104.5369,96.1012,104.5995,94.7579,86.6505,98.6710,98.8395,105.2203,96.7377,96.1954
2000-01-05,105.0049,104.9559,96.5425,93.0859,100.4468,82.1252,85.2790,74.6820,101.5541,106.2992,...,103.5898,95.2550,104.6054,94.6686,86.8567,99.0696,99.0366,105.4202,96.7848,96.3343
2000-01-06,104.9000,104.7967,96.1119,94.1179,100.2381,81.5353,85.0125,74.6106,101.8813,105.9962,...,102.4936,95.1679,104.6323,95.2204,86.8995,99.3596,97.9818,104.7016,96.7965,96.3733
2000-01-07,104.8932,105.1856,96.2838,94.2959,100.1559,81.5587,84.3793,75.2305,101.8659,105.9922,...,102.3636,95.7807,104.6382,95.6781,86.6741,98.8713,98.7276,104.9930,96.3979,96.0965
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-03-22,186.0643,127.3154,101.6661,179.5079,176.6998,186.9651,102.1772,206.4945,169.1713,92.7388,...,53.2585,98.9182,1990.7992,101.1395,81.4657,81.6287,117.3023,115.5920,88.9787,150.5894
2023-03-23,186.9954,128.6075,102.0388,179.3150,178.3258,189.4784,102.2634,204.2008,169.2580,93.2119,...,53.4884,99.4734,1995.4896,101.5455,81.2439,81.7538,118.5121,115.7257,89.6273,151.8991
2023-03-24,186.5218,128.2104,101.5374,177.9101,176.0068,186.5775,101.8825,206.3163,169.0571,93.2613,...,53.5274,99.6552,2003.9304,101.0757,80.6745,81.3933,118.2431,115.4804,88.8307,150.4140


In [8]:
df_fx_spot

,NZD,CZK,RUB,COP,IDR,HKD,ILS,PHP,HUF,NOK,...,JPY,AUD,CAD,PLN,KRW,MYR,ZAR,CLP,MXN,TWD
date,,,,,,,,,,,,,,,,,,,,,
2000-01-03,0.5250,0.028420,0.03656,0.053260,0.142040,0.12860,0.242840,0.025010,0.00399,0.1268,...,0.009857,0.6585,0.6912,0.2430,0.08869,0.2632,0.163090,0.18939,0.10637,0.031860
2000-01-04,0.5194,0.028350,0.03636,0.052840,0.140350,0.12858,0.243010,0.025140,0.00405,0.1258,...,0.009685,0.6550,0.6884,0.2423,0.08910,0.2632,0.164200,0.18888,0.10448,0.032710
2000-01-05,0.5202,0.028470,0.03669,0.052350,0.139950,0.12857,0.240890,0.024910,0.00406,0.1259,...,0.009604,0.6579,0.6897,0.2426,0.08814,0.2632,0.164790,0.18867,0.10521,0.032500
2000-01-06,0.5146,0.028480,0.03642,0.052290,0.138400,0.12857,0.240600,0.024660,0.00406,0.1259,...,0.009503,0.6533,0.6850,0.2430,0.08724,0.2632,0.164510,0.18975,0.10438,0.032410
2000-01-07,0.5175,0.028530,0.03642,0.052600,0.139560,0.12858,0.239840,0.024760,0.00404,0.1254,...,0.009493,0.6549,0.6869,0.2449,0.08791,0.2632,0.165080,0.19063,0.10550,0.032420
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-03-22,0.6222,0.045791,0.01298,0.020940,NaN,0.12743,0.275582,0.018334,0.00280,0.0956,...,0.007608,0.6685,0.7283,0.2316,0.07649,0.2244,0.054695,0.12353,0.05374,0.032772
2023-03-23,0.6250,0.045870,0.01315,0.021080,NaN,0.12740,0.277536,0.018412,0.00283,0.0963,...,0.007643,0.6684,0.7291,0.2313,0.07823,0.2263,0.055253,0.12399,0.05382,0.032952
2023-03-24,0.6203,0.045438,0.01294,0.021147,0.065994,0.12739,0.279036,0.018412,0.00280,0.0954,...,0.007650,0.6645,0.7276,0.2295,0.07732,0.2258,0.055075,0.12341,0.05422,0.032983
